# Step 1
- Create network of characters
- Calculate centrality measures
- Detect communities

In [33]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rcParams
from pyvis.network import Network

In [34]:
# SET VIZ TEMPLATE
# colors
YELLOW='yellow'
BLACK='#1a1a1a'
GREY='#696969'
WHITE='white'

# fonts
futura_font_path = 'graphfonts/Futura/futura-medium-condensed.ttf'
FUTURA = fm.FontProperties(fname=futura_font_path)

paypal_font_path = 'graphfonts/Paypal/PayPalSansSmall-Regular.ttf'
PAYPAL = fm.FontProperties(fname=paypal_font_path)

DPI=200

In [35]:
sheets = ['episode1', 'episode2', 'episode3', 'episode4', 'episode5', 'episode6']
data = pd.concat([pd.read_excel('data.xlsx', sheet_name = sheet) for sheet in sheets])
data

In [36]:
# clean data
df = (data
 .assign(sorted_pairs=lambda df_: df_.apply(lambda df_: '-'.join(sorted([df_['initiator'], df_['responder']])), axis=1))
 .assign(weight=lambda df_: df_.groupby(['sorted_pairs'])['sorted_pairs'].transform('count'))
 .assign(dialogue=lambda df_: df_.groupby(['sorted_pairs'])['dialogue'].transform(lambda df_: ' '.join(df_.unique())))
 .drop_duplicates('sorted_pairs')
 .drop(columns='sorted_pairs')
)
df

In [37]:
# create network
G = nx.from_pandas_edgelist(df, source='initiator', target='responder', edge_attr=['dialogue', 'weight'])

In [38]:
# show nodes in network
G.nodes

In [39]:
# show edges
G.edges

In [40]:
# count edges
len(G.edges)

In [41]:
# see network
net = Network(notebook=True, cdn_resources='remote', width='1000px', height='700px', bgcolor=BLACK, font_color=YELLOW)

node_degree = dict(G.degree)

# set node size attribute
nx.set_node_attributes(G, node_degree, 'size')

net.from_nx(G)
net.show('TWL.html')

# Centrality Measures

In [42]:
# show degree centrality
degree_df = pd.DataFrame.from_dict(nx.degree_centrality(G), orient='index', columns=['degree']).sort_values('degree', ascending=True)
degree_df

In [43]:
# plot degree distribution
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), facecolor=BLACK, dpi=DPI)
ax1.tick_params(axis='both', colors=YELLOW, labelsize=10)
ax1.set_facecolor(BLACK)
ax1.set_title('Empirical Cumulative Distribution Function', fontproperties=FUTURA, color=WHITE)
ax1.set_xlabel('Value', fontproperties=PAYPAL, color=WHITE, fontsize=8)
ax1.set_ylabel('Probability (at or below value)', fontproperties=PAYPAL, color=WHITE, fontsize=8)

ax2.tick_params(axis='both', colors=YELLOW, labelsize=10)
ax2.set_facecolor(BLACK)
ax2.set_title('Histogram', fontproperties=FUTURA, color=WHITE)
ax2.set_xlabel('Value', fontproperties=PAYPAL, color=WHITE, fontsize=8)
ax2.set_ylabel('Density', fontproperties=PAYPAL, color=WHITE, fontsize=8)

for label in ax1.get_xticklabels()+ax2.get_xticklabels():
    label.set_fontproperties(PAYPAL)
    label.set_fontsize(10)
    label.set_color(YELLOW)

for label in ax1.get_yticklabels():
    label.set_fontproperties(PAYPAL)
    label.set_fontsize(10)
    label.set_color(YELLOW)

for label in ax2.get_yticklabels():
    label.set_fontproperties(PAYPAL)
    label.set_fontsize(10)
    label.set_color(YELLOW)

# set spine color
for ax in [ax1, ax2]:
    ax.spines['bottom'].set_color(WHITE)
    ax.spines['left'].set_color(WHITE)

# Set the visibility of the spines on the top and left to off
for ax in [ax1, ax2]:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Calculate the ECDF and plot it on the left subplot
x = np.sort(degree_df)
y = np.arange(1, len(degree_df)+1) / len(degree_df)
ax1.plot(x, y, marker='h', linestyle='none', color=YELLOW)

# Plot the histogram on the right subplot
ax2.hist(degree_df, bins=10, density=True, color=YELLOW)

# Add a centered supertitle to represent both plots
fig.suptitle('Distribution of node degree', fontsize=18, fontweight='bold', fontproperties=FUTURA, color=WHITE, y=1.05)

# Show the plot
plt.show()

ECDF stands for Empirical Cumulative Distribution Function. It is a non-parametric estimator of the cumulative distribution function (CDF) of a random variable based on a sample of the data.

To construct the ECDF, we first sort the observations in our sample from smallest to largest. Then, for each observation, we calculate the proportion of the observations that are less than or equal to it, which gives us the cumulative probability of that observation. Finally, we plot these cumulative probabilities against the sorted observations to create the ECDF.

The ECDF is a useful tool for visualizing the distribution of a dataset, especially when we don't know the underlying distribution of the data. It provides a way to estimate the probability of observing a value less than or equal to a particular value in the data, which can be used for statistical inference, hypothesis testing, and model selection.

In [44]:
def plot_bar(df, sort_value, title):
    fig, ax = plt.subplots(figsize=(4,3), dpi=DPI, facecolor=BLACK)

    plt.title(title, fontproperties=FUTURA, color=WHITE, fontsize=9)

    ax.set_facecolor(BLACK)

    for label in ax.get_xticklabels():
        label.set_fontproperties(PAYPAL)
        label.set_fontsize(7)
        label.set_color(WHITE)

    for label in ax.get_yticklabels():
        label.set_fontproperties(FUTURA)
        label.set_fontsize(5.5)
        label.set_color(WHITE)
        
    for side in 'bottom,left'.split(','):
        ax.spines[side].set_edgecolor(WHITE)
        
    for side in 'top,right'.split(','):
        ax.spines[side].set_visible(False)

    plot = df.sort_values(sort_value, ascending=False)[0:10].plot.barh(ax=ax, legend=False, color=YELLOW)

    for i, value in enumerate(df.sort_values(sort_value, ascending=False)[0:10][sort_value]):
        ax.text(value, i, str(round(value,2)), fontproperties=PAYPAL, color=WHITE, va='center', fontsize=5.5)

    return plot

plot_bar(degree_df, sort_value='degree', title='Degree centrality distribution (top 10)');

In [45]:
# show degree centrality
closeness_df = pd.DataFrame.from_dict(nx.closeness_centrality(G), orient='index', columns=['closeness'])
plot_bar(closeness_df, sort_value='closeness', title='Closeness centrality distribution (top 10)');

In [46]:
# show betweeness centrality
betweenness_df = pd.DataFrame.from_dict(nx.betweenness_centrality(G), orient='index', columns=['betweenness'])
plot_bar(betweenness_df, sort_value='betweenness', title='Betweenness centrality distribution (top 10)');

In [47]:
# show eigenvector centrality
eigenvector_df = pd.DataFrame.from_dict(nx.eigenvector_centrality(G), orient='index', columns=['eigenvector'])
plot_bar(eigenvector_df, sort_value='eigenvector', title='Eigenvector centrality distribution (top 10)');

# 

# Clustering coefficient, average path length

In [48]:
# calculate average shortest path length (apl)
apl = nx.average_shortest_path_length(G, weight='weight')
apl

In [49]:
# calculate clustering coefficient (cc)
cc = nx.clustering(G, weight='weight')
cc

In [50]:
from tqdm import tqdm

def average_path_length_disconnected(G):
    connected_components = [G.subgraph(c) for c in nx.connected_components(G)]
    avg_path_lengths = [nx.average_shortest_path_length(cc) for cc in connected_components]
    overall_avg_path_length = sum(avg_path_lengths) / len(connected_components)
    return overall_avg_path_length

In [51]:
nx.node_connected_component(G, 'NURSE')

In [52]:
small_world_measures = []

for p in tqdm(range(0, 10, 1)):
    p = p/10 # rewiring rate
    small_world_graph = nx.watts_strogatz_graph(len(G.nodes), 6, p, seed=1)
    cc = nx.average_clustering(small_world_graph)
    apl = average_path_length_disconnected(small_world_graph)
    small_world_measures.append([p, cc, apl])

In [53]:
small_world_measures

In [54]:
# Clustering coefficient
plt.scatter([item[0] for item in small_world_measures], [item[1] for item in small_world_measures])
plt.xlabel("p")
plt.ylabel("Clustering Coefficient")
plt.show()

In [55]:
len(G.nodes)

In [56]:
import networkx as nx

# Load your network data as a NetworkX graph
# G = nx.read_edgelist('your_network_data.edgelist')

# Calculate the number of nodes and edges in the graph
n = G.number_of_nodes()
m = G.number_of_edges()

# Calculate the average degree of the graph
k = 2 * m / n

# Generate a Watts-Strogatz graph with the same number of nodes, 
# the same average degree, and the same number of edges as the original graph
p = 0.1  # Rewiring probability
WS = nx.watts_strogatz_graph(n, int(k), p)

# Calculate the clustering coefficient and average shortest path length of both graphs
cc_orig = nx.average_clustering(G)
cc_WS = nx.average_clustering(WS)
l_orig = nx.average_shortest_path_length(G)
l_WS = nx.average_shortest_path_length(WS)

# Print the results
print(f'Clustering coefficient of original graph: {cc_orig}')
print(f'Clustering coefficient of Watts-Strogatz graph: {cc_WS}')
print(f'Average shortest path length of original graph: {l_orig}')
print(f'Average shortest path length of Watts-Strogatz graph: {l_WS}')

# Check if the original graph has a higher clustering coefficient and similar average shortest path length 
# compared to the Watts-Strogatz graph, indicating a small world network
if cc_orig > cc_WS and l_orig <= l_WS:
    print('The graph may follow a small world network.')
else:
    print('The graph does not follow a small world network.')


In [57]:
len(G.edges)

In [58]:
from tqdm import tqdm

n = G.number_of_nodes()
m = G.number_of_edges()

# Calculate the average degree of the graph
k = 2 * m / n

small_world_measures = []

for p in tqdm(range(0, 10, 1)):
    p = p/10 # rewiring rate
    small_world_graph = nx.watts_strogatz_graph(n, int(k), p, seed=1)
    cc = nx.average_clustering(small_world_graph)
    apl = nx.average_shortest_path_length(small_world_graph)
    small_world_measures.append({'rewiring rate': p, 'clustering coefficient': cc, 'average path length': apl})

small_world_measures


In [59]:
def plot_scatter(x, y, x_label, y_label):
    fig, ax = plt.subplots(figsize=(3,2.5), dpi=DPI, facecolor=BLACK)
    ax.tick_params(axis='both', colors=YELLOW, labelsize=10)

    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontproperties(PAYPAL)
        label.set_fontsize(7)
        label.set_color(YELLOW)

    for side in 'bottom,left'.split(','):
        ax.spines[side].set_edgecolor(WHITE)

    for side in 'top,right'.split(','):
        ax.spines[side].set_visible(False)

    ax.set_facecolor(BLACK)
    ax.scatter(x, y, c=YELLOW, s=12, marker='h')
    ax.set_xlabel(x_label, fontproperties=PAYPAL, color=WHITE, fontsize=6)
    ax.set_ylabel(y_label, fontproperties=PAYPAL, color=WHITE, fontsize=6)
    plt.show()

In [60]:
# see if network looks like small world network    
x = [item['rewiring rate'] for item in small_world_measures]
y = [item['clustering coefficient'] for item in small_world_measures]
plot_scatter(x, y, 'p', 'Clustering Coefficient')

In [61]:
plot_scatter(
    [item['rewiring rate'] for item in small_world_measures],
    [item['average path length'] for item in small_world_measures],
    'p',
    'Average Path Length'
)

The network follows the small world network graph properties.

# Community detection

In [62]:
import community as community_louvain

# create communities
communities = community_louvain.best_partition(G)

# set communities as node
nx.set_node_attributes(G, communities, 'group')

# show network
com_net = Network(notebook=True, width='1000px', height='700px', bgcolor=BLACK, font_color=YELLOW)
com_net.from_nx(G)
com_net.show('TWL_com.html')